In [ ]:
import set_path

import numpy as np
import laspy
import geopandas as gpd
import shapely.geometry as sg

from upcp.utils import ahn_utils
from upcp.utils.interpolation import FastGridInterpolator
from upcp.region_growing.label_connected_comp import LabelConnectedComp

import gvl.helper_functions as helpers
from gvl.tree_detector import DetectorTree

In [ ]:
import warnings  # temporary, to supress deprecationwarnings from shapely
warnings.filterwarnings('ignore')

In [ ]:
# AHN classification
AHN_OTHER = 1
AHN_GROUND = 2
AHN_BUILDING = 6
AHN_WATER = 9
AHN_ARTIFACT = 26

In [ ]:
# Our classification
UNKNOWN = 0
NOISE = 1
TREE = 2
OTHER = 3

In [ ]:
# Use AHN subtiles
BASE_FOLDER = '/media/dbloembergen/PointCloud/AHN4'

tilecode = '117_484'

las_file = f'{BASE_FOLDER}/AMS_subtiles_1000/ahn4_{tilecode}.laz'
tree_ref_file = '../datasets/validation/joined_trees_bgt_gissib_1_5_amsterdam.shp' # TODO get geovisia dataset
ahn_dtm_folder = f'{BASE_FOLDER}/npz_subtiles_1000/'

ahn_reader = ahn_utils.NPZReader(ahn_dtm_folder, caching=False)

In [ ]:
las = laspy.read(las_file)
points_xyz = np.vstack((las.x, las.y, las.z)).T
bbox = las.header.min[0], las.header.min[1], las.header.max[0], las.header.max[1]

In [ ]:
tree_gdf = gpd.read_file(tree_ref_file, crs='epsg:28992')

trees_in_bbox = tree_gdf[tree_gdf.within(sg.box(*bbox, ccw=True))]
tree_points = list(trees_in_bbox['geometry'].values)

## Reduce the point cloud
### Filter 1: scalar fields
- classification -> 'overig label' points

We dont want to filter in the number_of_returns scalar field. It will remove too much valuable tree points. 

In [ ]:
mask = (las.classification == AHN_OTHER)

### Filter 2: remove points close to ground

In [ ]:
# Minimum height above ground in meters
MIN_HAG = 2.5

# Get ground height
ground_z = ahn_reader.interpolate(helpers.get_tilecode_from_filename(las_file),
                                  points_xyz[mask])

height_mask = (points_xyz[mask, 2] - ground_z >= MIN_HAG) | np.isnan(ground_z)

points_xyz = points_xyz[mask][height_mask]
points_i = las.intensity[mask][height_mask]
ground_z = ground_z[height_mask]

## Label the point cloud
### Label 1: label "tree" clusters based on ground truth tree points

In [ ]:
# TODO: progress indication
grid_size = 0.6          # TODO
min_component_size = 50  # TODO
min_height = 3.5         # TODO

lcc = LabelConnectedComp(grid_size=grid_size,
                         min_component_size=min_component_size)
point_components = lcc.get_components(points_xyz)

# Label tree like clusters
tree_mask = helpers.label_tree_like_components(
                                points_xyz, ground_z, point_components,
                                tree_points, min_height)

### Label 2: label "noise" points

In [ ]:
grid_size = 0.9          # TODO
min_component_size = 50  # TODO

mask_ids = np.where(~tree_mask)[0] # Only the possible noise points

lcc = LabelConnectedComp(grid_size=grid_size,
                         min_component_size=min_component_size)
point_components = lcc.get_components(points_xyz[mask_ids])

In [ ]:
# Noise filter
noise_pts = point_components == -1
print(f'Found {np.count_nonzero(noise_pts)} noise points in '
             + f'clusters < {min_component_size} points.')

noise_mask = np.zeros((len(points_xyz),), dtype=bool)
noise_mask[mask_ids] = noise_pts

### Label 3: label "non-tree" clusters based on normal values and HAG

In [ ]:
cc_labels = np.unique(point_components)
cc_labels = set(cc_labels).difference((-1,))

normals = helpers.calculate_normals(points_xyz)
normals_mask = np.zeros(len(normals[:,2]), dtype=bool)

# Height above ground
hag = points_xyz[:,2] - ground_z
hag_mask = np.zeros(len(points_xyz[:,2]), dtype=bool)

# Iterate over the clusters
for cc in cc_labels:
    # select points that belong to the cluster
    cc_mask_jm = (point_components == cc)

    # If most of the points point up, it's not a tree.
    if np.abs(normals[:,2][mask_ids[cc_mask_jm]]).mean() > 0.85:  # TODO
        normals_mask[mask_ids[cc_mask_jm]] = True
    
    # TODO come up with something clever for x/y flatness
    # Do a similar thing with the x normals # TODO Daan validate
    # elif normals[:,0][mask_ids[cc_mask_jm]].mean() < 0.03 and normals[:,0][mask_ids[cc_mask_jm]].mean() > -0.03:
    #     normals_mask[mask_ids[cc_mask_jm]] = True
    
    # TODO look at shape of cluster, e.g. minimum bounding rectangle + min_width check.
        
    # If the tree is smaller than 2.5 meters or higher than 38 (TODO) meters, it's not a tree.
    elif hag[mask_ids[cc_mask_jm]].max() > 38 or hag[mask_ids[cc_mask_jm]].max() < 2.5:
        hag_mask[mask_ids[cc_mask_jm]] = True    

In [ ]:
labels = np.ones((len(points_xyz),), dtype='uint16') * UNKNOWN
labels[tree_mask] = TREE
labels[noise_mask] = NOISE
labels[normals_mask] = OTHER
labels[hag_mask] = OTHER

## Save the point cloud

In [ ]:
header = laspy.LasHeader(point_format=3, version="1.2")
header.offsets = las.header.offsets
header.scales = las.header.scales

new_las = laspy.LasData(header)

new_las.x = points_xyz[:, 0]
new_las.y = points_xyz[:, 1]
new_las.z = points_xyz[:, 2]
new_las.intensity = points_i

new_las.add_extra_dim(laspy.ExtraBytesParams(name="label", type="uint16",
                                         description="Label"))
new_las.label = labels

new_las.add_extra_dim(laspy.ExtraBytesParams(name="normal_x", type="float",
                  description="normal_x"))
new_las.add_extra_dim(laspy.ExtraBytesParams(name="normal_y", type="float",
                  description="normal_y"))    
new_las.add_extra_dim(laspy.ExtraBytesParams(name="normal_z", type="float",
                  description="normal_z"))
new_las.normal_x = normals[:,0]
new_las.normal_y = normals[:,1]
new_las.normal_z = normals[:,2]

new_las.write(f'../datasets/pointcloud/tree_{tilecode}.laz')